In [19]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [9]:
train = pd.read_csv("../data/train.csv").drop(columns=['ID'])
test = pd.read_csv("../data/test.csv").drop(columns=['ID'])

In [11]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [13]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [15]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [21]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [23]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [25]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [27]:
# 전처리된 훈련 데이터와 테스트 데이터를 CSV 파일로 저장
X_train_encoded['임신 성공 여부'] = y  # 타겟 변수도 훈련 데이터에 포함시킴
X_train_encoded.to_csv('../data/train_encoded.csv', index=False)  # 훈련 데이터 저장

X_test_encoded.to_csv('../data/test_encoded.csv', index=False)  # 테스트 데이터 저장

print("전처리된 데이터가 성공적으로 저장되었습니다.")

전처리된 데이터가 성공적으로 저장되었습니다.


### Train

In [56]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

### Predict

In [10]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [11]:
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [13]:
sample_submission.to_csv('../submission/baseline_submit.csv', index=False)

In [ ]:
#앙상븡 모델 적용 후 성능평가

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
import lightgbm as lgb

# 전처리된 훈련 데이터와 테스트 데이터를 파일에서 로드
X_train_encoded = pd.read_csv('../data/train_encoded.csv')
X_test_encoded = pd.read_csv('../data/test_encoded.csv')

# 타겟 변수와 특성 분리
y = X_train_encoded['임신 성공 여부']
X = X_train_encoded.drop('임신 성공 여부', axis=1)

# 훈련 데이터 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의 (여러 모델을 사용)
et_model = ExtraTreesClassifier(random_state=42)
rf_model = RandomForestClassifier(random_state=42)
xgb_model = xgb.XGBClassifier(random_state=42)
lgb_model = lgb.LGBMClassifier(random_state=42)

# 앙상블 모델 정의 (Voting Classifier)
voting_model = VotingClassifier(estimators=[
    ('et', et_model),
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lgb', lgb_model)
], voting='soft')  # soft voting: 예측 확률의 평균으로 최종 예측

# 모델 학습
voting_model.fit(X_train, y_train)

# 예측
y_pred = voting_model.predict(X_val)
y_pred_proba = voting_model.predict_proba(X_val)[:, 1]  # 예측 확률

# 성능 평가
print("Confusion Matrix")
print(confusion_matrix(y_val, y_pred))
print("\nClassification Report")
print(classification_report(y_val, y_pred))

# 예측 결과를 sample_submission 파일에 저장
pred_proba = voting_model.predict_proba(X_test_encoded)[:, 1]
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['probability'] = pred_proba

# 제출 파일로 저장
sample_submission.to_csv('../submission/baseline_submit.csv', index=False)

print("예측 결과가 'baseline_submit.csv'에 저장되었습니다.")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026711 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 716
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521
Confusion Matrix
[[35869  2276]
 [11035  2091]]

Classification Report
              precision    recall  f1-score   support

           0       0.76      0.94      0.84     38145
           1       0.48      0.16      0.24     13126

    accuracy                           0.74     51271
   macro avg       0.62      0.55      0.54     51271
weighted avg      

smote 적용 코드(아주 조금 오름)

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import lightgbm as lgb

# 데이터 로드
X_train_encoded = pd.read_csv('../data/train_encoded.csv')
X_test_encoded = pd.read_csv('../data/test_encoded.csv')

# 타겟 변수와 특성 분리
y = X_train_encoded['임신 성공 여부']
X = X_train_encoded.drop('임신 성공 여부', axis=1)

# 훈련 데이터와 검증 데이터 나누기 (테스트 데이터는 원본 그대로 둠)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# SMOTE 적용 (훈련 데이터에만 오버샘플링 적용)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 데이터 크기 조정 (필요시)
target_size = 150000
if len(X_train_resampled) > target_size:
    X_train_resampled = X_train_resampled[:target_size]
    y_train_resampled = y_train_resampled[:target_size]

# 개별 모델 정의
et_model = ExtraTreesClassifier(random_state=42)
rf_model = RandomForestClassifier(random_state=42)
xgb_model = xgb.XGBClassifier(random_state=42)
lgb_model = lgb.LGBMClassifier(random_state=42)

# 개별 모델 하이퍼파라미터 튜닝
param_grid_et = {'n_estimators': [100, 200]}
grid_et = GridSearchCV(et_model, param_grid_et, cv=3, n_jobs=4, verbose=2)
grid_et.fit(X_train_resampled, y_train_resampled)

param_grid_rf = {'n_estimators': [100, 200]}
grid_rf = GridSearchCV(rf_model, param_grid_rf, cv=3, n_jobs=4, verbose=2)
grid_rf.fit(X_train_resampled, y_train_resampled)

param_grid_xgb = {'max_depth': [3, 5]}
grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, n_jobs=4, verbose=2)
grid_xgb.fit(X_train_resampled, y_train_resampled)

param_grid_lgb = {'num_leaves': [31, 50]}
grid_lgb = GridSearchCV(lgb_model, param_grid_lgb, cv=3, n_jobs=4, verbose=2)
grid_lgb.fit(X_train_resampled, y_train_resampled)

# 최적의 하이퍼파라미터로 학습된 모델로 VotingClassifier 생성
voting_model = VotingClassifier(estimators=[
    ('et', grid_et.best_estimator_),
    ('rf', grid_rf.best_estimator_),
    ('xgb', grid_xgb.best_estimator_),
    ('lgb', grid_lgb.best_estimator_)
], voting='soft')

# 앙상블 모델 학습
voting_model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 예측
y_pred = voting_model.predict(X_val)
y_pred_proba = voting_model.predict_proba(X_val)[:, 1]  # 예측 확률

# 성능 평가
print("Confusion Matrix")
print(confusion_matrix(y_val, y_pred))
print("\nClassification Report")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측 및 제출 파일 생성
pred_proba = voting_model.predict_proba(X_test_encoded)[:, 1]
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['probability'] = pred_proba

# CSV 파일로 저장
sample_submission.to_csv('../submission/baseline_submit.csv', index=False)
print("예측 결과가 'baseline_submit.csv'에 저장되었습니다.")

In [ ]:
테스트 데이터 비율 5:5

In [ ]:
#무지성 총공격

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import optuna

# 데이터 로드
X_train_encoded = pd.read_csv('../data/train_encoded.csv')
X_test_encoded = pd.read_csv('../data/test_encoded.csv')

# 타겟 변수와 특성 분리
y = X_train_encoded['임신 성공 여부']
X = X_train_encoded.drop('임신 성공 여부', axis=1)

# 데이터 분할 (훈련 / 검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 스케일링 (딥러닝 모델을 위해 정규화)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test_encoded)

print("✅ 데이터 로드 및 전처리 완료!")

✅ 데이터 로드 및 전처리 완료!


In [10]:
import optuna
import xgboost as xgb
import lightgbm as lgb


# LightGBM 하이퍼파라미터 최적화
def objective_lgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),  # 변경됨
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # 변경됨
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)  # 변경됨
    }
    
    model = lgb.LGBMClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_val, y_val)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=200)
best_lgb_params = study_lgb.best_params
print(f"🔥 최적 LightGBM 하이퍼파라미터: {best_lgb_params}")

[I 2025-02-18 02:29:53,136] A new study created in memory with name: no-name-24a7f11c-60cb-42d2-8d46-926c4a93214d
[I 2025-02-18 02:29:56,651] Trial 0 finished with value: 0.7457237034580952 and parameters: {'n_estimators': 169, 'max_depth': 7, 'learning_rate': 0.04025297364106417, 'subsample': 0.7358166445724317, 'colsample_bytree': 0.8102269475077959}. Best is trial 0 with value: 0.7457237034580952.
[I 2025-02-18 02:30:01,916] Trial 1 finished with value: 0.7403015349807883 and parameters: {'n_estimators': 416, 'max_depth': 5, 'learning_rate': 0.22208770832618788, 'subsample': 0.6183057381008377, 'colsample_bytree': 0.5133767447974953}. Best is trial 0 with value: 0.7457237034580952.
[I 2025-02-18 02:30:03,334] Trial 2 finished with value: 0.7450020479413314 and parameters: {'n_estimators': 101, 'max_depth': 4, 'learning_rate': 0.06805186805938937, 'subsample': 0.8343857611064573, 'colsample_bytree': 0.6422163723864018}. Best is trial 0 with value: 0.7457237034580952.
[I 2025-02-18 02

🔥 최적 XGBoost 하이퍼파라미터: {'n_estimators': 250, 'max_depth': 4, 'learning_rate': 0.09681040383979617, 'subsample': 0.7481684127880879, 'colsample_bytree': 0.9995661813659329}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017703 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573


[I 2025-02-18 02:31:24,426] Trial 0 finished with value: 0.7434807200951805 and parameters: {'n_estimators': 111, 'num_leaves': 85, 'learning_rate': 0.17264050558059515, 'subsample': 0.6452574686688055, 'colsample_bytree': 0.7180151039239253}. Best is trial 0 with value: 0.7434807200951805.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024190 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573


[I 2025-02-18 02:31:26,799] Trial 1 finished with value: 0.745119073160266 and parameters: {'n_estimators': 161, 'num_leaves': 90, 'learning_rate': 0.02629543092828785, 'subsample': 0.8070970900208738, 'colsample_bytree': 0.8684839212354779}. Best is trial 1 with value: 0.745119073160266.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023860 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573


[I 2025-02-18 02:53:00,998] Trial 198 finished with value: 0.7456456866454721 and parameters: {'n_estimators': 459, 'num_leaves': 88, 'learning_rate': 0.011294585384438904, 'subsample': 0.8473039714260397, 'colsample_bytree': 0.581905227587702}. Best is trial 92 with value: 0.746367342162236.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027690 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573


[I 2025-02-18 02:53:08,840] Trial 199 finished with value: 0.7459772580991204 and parameters: {'n_estimators': 490, 'num_leaves': 74, 'learning_rate': 0.012558651577570582, 'subsample': 0.8675635208346766, 'colsample_bytree': 0.5687567412233688}. Best is trial 92 with value: 0.746367342162236.


🔥 최적 LightGBM 하이퍼파라미터: {'n_estimators': 492, 'num_leaves': 88, 'learning_rate': 0.014666949438705097, 'subsample': 0.8766412273868395, 'colsample_bytree': 0.5848756135604947}


In [11]:
import optuna
from sklearn.ensemble import RandomForestClassifier

# XGBoost 하이퍼파라미터 최적화
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),  # 변경됨
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # 변경됨
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)  # 변경됨
    }
    
    model = xgb.XGBClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_val, y_val)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=200)
best_xgb_params = study_xgb.best_params
print(f"🔥 최적 XGBoost 하이퍼파라미터: {best_xgb_params}")

# 랜덤 포레스트 하이퍼파라미터 최적화
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
    }
    
    model = RandomForestClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_val, y_val)

# Optuna 스터디 생성
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=200)
best_rf_params = study_rf.best_params
print(f"🔥 최적 Random Forest 하이퍼파라미터: {best_rf_params}")

[I 2025-02-18 02:54:48,255] A new study created in memory with name: no-name-c2ee9916-5849-441c-ba6f-77a93ad672fe
[I 2025-02-18 02:54:55,625] Trial 0 finished with value: 0.745119073160266 and parameters: {'n_estimators': 461, 'max_depth': 9, 'learning_rate': 0.012873750181589297, 'subsample': 0.6184643105349974, 'colsample_bytree': 0.5921870384176341}. Best is trial 0 with value: 0.745119073160266.
[I 2025-02-18 02:54:58,060] Trial 1 finished with value: 0.7451775857697334 and parameters: {'n_estimators': 153, 'max_depth': 7, 'learning_rate': 0.0210250488240244, 'subsample': 0.5432974095813292, 'colsample_bytree': 0.7455797821607777}. Best is trial 1 with value: 0.7451775857697334.
[I 2025-02-18 02:55:03,106] Trial 2 finished with value: 0.7457237034580952 and parameters: {'n_estimators': 322, 'max_depth': 7, 'learning_rate': 0.027046905842027413, 'subsample': 0.7945150914039159, 'colsample_bytree': 0.6306238463192702}. Best is trial 2 with value: 0.7457237034580952.
[I 2025-02-18 02:

🔥 최적 XGBoost 하이퍼파라미터: {'n_estimators': 359, 'max_depth': 5, 'learning_rate': 0.045093620098282834, 'subsample': 0.6669970717173888, 'colsample_bytree': 0.934119080218835}


InvalidParameterError: The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.

In [12]:
import optuna
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 하이퍼파라미터 최적화
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),  # 수정된 부분
    }
    
    model = RandomForestClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_val, y_val)

# Optuna 스터디 생성
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=20)
best_rf_params = study_rf.best_params
print(f"🔥 최적 Random Forest 하이퍼파라미터: {best_rf_params}")

[I 2025-02-18 03:15:37,585] A new study created in memory with name: no-name-748e1273-70cd-4b9c-a12a-a4a2a5c6720c
[I 2025-02-18 03:16:33,558] Trial 0 finished with value: 0.7416473249985371 and parameters: {'n_estimators': 136, 'max_depth': 4, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 0 with value: 0.7416473249985371.
[I 2025-02-18 03:18:30,335] Trial 1 finished with value: 0.7442608882214117 and parameters: {'n_estimators': 163, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_features': None}. Best is trial 1 with value: 0.7442608882214117.
[I 2025-02-18 03:19:26,608] Trial 2 finished with value: 0.7416473249985371 and parameters: {'n_estimators': 135, 'max_depth': 4, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 1 with value: 0.7442608882214117.
[I 2025-02-18 03:20:13,793] Trial 3 finished with value: 0.7422324510932106 and parameters: {'n_estimators': 328, 'max_depth': 9, 'min_sample

🔥 최적 Random Forest 하이퍼파라미터: {'n_estimators': 215, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}


In [15]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

# 최적 하이퍼파라미터로 개별 모델 정의
rf_model = RandomForestClassifier(
    n_estimators=215,
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=8,
    max_features=None,
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    n_estimators=359,
    max_depth=5,
    learning_rate=0.045093620098282834,
    subsample=0.6669970717173888,
    colsample_bytree=0.934119080218835,
    random_state=42
)

lgb_model = lgb.LGBMClassifier(
    n_estimators=492,
    num_leaves=88,
    learning_rate=0.014666949438705097,
    subsample=0.8766412273868395,
    colsample_bytree=0.5848756135604947,
    random_state=42
)

# Stacking 앙상블 정의
stacking_model = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgb', lgb_model)
    ],
    final_estimator=LogisticRegression(),
    cv=3
)

# 모델 학습
stacking_model.fit(X_train, y_train)
print("✅ Stacking 모델 학습 완료!")

# 검증 데이터 예측
y_pred = stacking_model.predict(X_val)

# 평가 지표 확인
from sklearn.metrics import confusion_matrix, classification_report
print("Confusion Matrix")
print(confusion_matrix(y_val, y_pred))
print("\nClassification Report")
print(classification_report(y_val, y_pred))


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35321, number of negative: 101399
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# PyTorch 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 데이터 변환
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)

# 모델 학습
for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("✅ 딥러닝 모델 학습 완료!")

Epoch 0, Loss: 0.6963029503822327
Epoch 10, Loss: 0.5769034028053284
Epoch 20, Loss: 0.5450228452682495
Epoch 30, Loss: 0.5260497331619263
Epoch 40, Loss: 0.5124301314353943
✅ 딥러닝 모델 학습 완료!


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# PyTorch 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 과정에서 검증 손실을 모니터링하는 방법
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val.values.reshape(-1, 1), dtype=torch.float32).to(device)

best_val_loss = float('inf')
patience = 5  # 조기 종료의 patience 설정
counter = 0

for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # 검증 데이터에서 손실 계산
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # 조기 종료
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

print("✅ 딥러닝 모델 학습 완료!")

Epoch 0, Train Loss: 0.7029680013656616, Validation Loss: 0.6860185861587524
Epoch 10, Train Loss: 0.5763489007949829, Validation Loss: 0.5762265920639038
Epoch 20, Train Loss: 0.5395788550376892, Validation Loss: 0.5381858944892883
Epoch 30, Train Loss: 0.5211194157600403, Validation Loss: 0.5207911133766174
Epoch 40, Train Loss: 0.5083341002464294, Validation Loss: 0.5086693167686462
Epoch 50, Train Loss: 0.5013975501060486, Validation Loss: 0.502371609210968
Epoch 60, Train Loss: 0.4975058436393738, Validation Loss: 0.49911773204803467
Epoch 70, Train Loss: 0.4954114258289337, Validation Loss: 0.4975021183490753
Epoch 80, Train Loss: 0.4937605857849121, Validation Loss: 0.4961315393447876
Epoch 90, Train Loss: 0.4924192428588867, Validation Loss: 0.49504372477531433
Epoch 100, Train Loss: 0.4913056492805481, Validation Loss: 0.49416810274124146
Epoch 110, Train Loss: 0.49032527208328247, Validation Loss: 0.4935409426689148
Epoch 120, Train Loss: 0.48945632576942444, Validation Loss:

In [16]:
# Stacking 모델 + 딥러닝 예측 결합
stacking_preds = stacking_model.predict_proba(X_test_encoded)[:, 1]

# 딥러닝 예측
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
dl_preds = model(X_test_tensor).cpu().detach().numpy().flatten()

# 평균 Ensemble
final_preds = (stacking_preds + dl_preds) / 2

# 제출 파일 생성
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['probability'] = final_preds
sample_submission.to_csv('../submission/baseline_submit2.csv', index=False)
print("✅ 예측 결과가 'baseline_submit2.csv'에 저장되었습니다.")

✅ 예측 결과가 'baseline_submit2.csv'에 저장되었습니다.
